In [1]:
from glob import glob
import os

In [2]:
texts = glob('TEXT2/*')
len(texts)

1499

In [3]:
texts[:5]

['TEXT2/app_4143_6285_phnd_deb-3.TextGrid',
 'TEXT2/app_4148_6296_phnd_deb-2.TextGrid',
 'TEXT2/app_4071_6141_phnd_deb-3.TextGrid',
 'TEXT2/app_4170_6339_phnd_deb-2.TextGrid',
 'TEXT2/app_4062_6123_phnd_deb-3.TextGrid']

In [4]:
audios = glob('audio/**/**.wav', recursive = True)
speakers = {}
for a in audios:
    speakers[os.path.split(a)[1].replace('.wav', '')] = a
    
len(speakers)

1500

In [5]:
import soundfile as sf
import textgrid

In [6]:
from tqdm import tqdm

directory = 'debate'

os.system(f'mkdir {directory}')
os.system(f'mkdir {directory}/wav')
os.system(f'mkdir {directory}/text')

256

In [7]:
def split(path):
    new_path = os.path.split(path)[1].replace('.TextGrid', '')
    y, sr = sf.read(speakers[new_path])
    index = 0
    path_textgrid = speakers[new_path]
    tg = textgrid.TextGrid.fromFile(path)
    for i in range(len(tg)):
        for k in range(len(tg[i])):
            try:
                splitted = y[int(sr * tg[i][k].minTime): int(sr * tg[i][k].maxTime)]
                sf.write(f'{directory}/wav/{new_path}-{i}-{k}.wav', splitted, sr)
                with open(f'{directory}/text/{new_path}-{i}-{k}.wav.txt', 'w') as fopen:
                    fopen.write(tg[i][k].mark)
            except Exception as e:
                print('nested exception', e)

In [8]:
split(texts[0])

In [9]:
wavs = glob(f'{directory}/wav/*.wav')
wavs[0]

'debate/wav/app_4221_6441_phnd_deb-1-0-59.wav'

In [10]:
import IPython.display as ipd
ipd.Audio(wavs[1])

In [11]:
!cat {wavs[1].replace('/wav/', '/text/') + '.txt'}

for (uh) brands and (uh) businesses to gain some sort of traction some sort reputation and I feel like that in a business point of view is

In [12]:
def loop(texts):
    texts = texts[0]
    for t in tqdm(texts):
        try:
            split(t)
        except Exception as e:
            print(e)
            pass

In [13]:
loop((texts[:1],))

100%|██████████| 1/1 [00:01<00:00,  1.43s/it]


In [ ]:
import mp

mp.multiprocessing(texts, loop, cores = 8, returned = False)

  9%|▊         | 16/187 [00:49<08:52,  3.11s/it]

In [ ]:
# !tar --use-compress-program="pigz -0 -k " -cf part5-debate.tar.gz debate